# RV BATCH FILE PROCESSING

In [1]:
# NOTA IMPORTANTE - CÓMO IMPLEMENTAR LO DEL "TIMEOUT".
# https://stackoverflow.com/questions/25027122/break-the-function-after-certain-time
# NO SIRVE - SOLO SIRVE PARA ENTORNOS UNIX

This notebook takes the GTO objects table and calculates the Lomb-Scargle periodogram for each of the RV curves, storing the results in `FITS` files with a primary header and two additional header data units, one with the RV curve itself and another with the periodogram.

Additionally, the GTO objects table and file is updated with new columns storing the results for each object.

Error control is implemented to prevent exceptions and timeouts to interrupt the processing. In case any of these errors happen, it is so recorded in the results table. Timeout value is dinamically updated, as the calculation for each record is completed.

**Calculation conditions:**

The stars currently under analysis are in the $He_{3}$ instability band, so they are expected to have effective temperatures in the range $T_{eff}\in[3300, 4300] K$, $\log g\in[4.5, 5.1]$, and masses in the range $M_{star}\in[0.20, 0.60] M_{Sun}$.

According to _Table 2_ in _The theoretical instability strip of M dwarf stars (Rodríguez-López, C., et al. 2014, MNRAS, 438, 2371)_ these stars have typical periods of $20 min$ to $3 h$, so we will set the limits of the periodogram frequencies for periods between $10 min$ and $5 h$. In days, this corresponds to a range of $P\in[0.006944, 0.208333] d$ or, equivalently, frequencies in the range $f\in[144, 4.8] d^{-1}$.

We will use the _Generalized Lomb Scargle Periodogram_ method, as described in [Zechmeister and Kürster, 2009](https://www.aanda.org/articles/aa/full_html/2009/11/aa11296-08/aa11296-08.html) and implemented by GitHub repository [mzechmeister/GLS](https://github.com/mzechmeister/GLS), using the default _ZK_ normalization.

## Modules and configuration

### Modules

In [2]:
# Modules import:
#from collections import OrderedDict
import pandas as pd
import numpy as np
import time

#from scipy import stats

# https://github.com/mzechmeister/GLS
from gls import Gls

from astropy.table import Table, QTable
#from astropy.timeseries import TimeSeries
from astropy import units as u
from astropy.io import fits
#from astropy.time import Time

import lightkurve as lk

#%matplotlib inline
import matplotlib.pyplot as plt

from pylab import rcParams
rcParams['figure.figsize'] = 11, 11

#import seaborn as sns
#sns.set_style("white", {'figure.figsize':(15,10)})
#sns.set_style("whitegrid")
#sns.set(rc={'figure.figsize':(15,8)})

### Configuration

In [3]:
# Configuration:
# Files and folders (WARNING: THIS FOLDER STRUCTURE MUST EXIST PREVIOUSLY):
GTO_FILE = "../data/GTO_objects_with_PG.csv"
IN_RV_FOLDER = "../data/CARMENES_GTO_RVs/"
OUT_IMG_FOLDER = "../data/CARMENES_GTO_RVs_PGs/figures/"
OUT_PROCESSED_FOLDER = "../data/CARMENES_GTO_RVs_PGs/"
#OUT_GTO_FILE = "../data/GTO_objects_withRVPG.csv"
#OUT_PG_FILE = "../data/GTO_PGs.csv"
#OUT_ERROR_FILE = "../data/GTO_PG_ERRORs.csv"

# Periodogram constants:
FBEG = 4.8
FEND = 144
PBEG = None # Default
PEND = None # Default
OFAC = 10 # Default
HIFAC = 1 # Default
FREQ = np.linspace(start=FBEG, stop=FEND, num=1000001) # 1,000,001 points between 4.8 and 144 d^(-1)
# Must be compatible with FBEG, FEND values
NORM = "ZK" # Default
LS = False # Default
FAST = False # Default
VERBOSE = False # Default
FAP_LEVELS_PLOT = [0.01, 0.05, 0.10] # FAP reference levels to plot

# Initial timeout value (to prevent a stuck file to interrupt the whole process)
#INITIAL_TIMEOUT = 600 # Seconds


In [4]:
def rv_load(filename: str):
    '''Load the RV file and returns a lightcurve object'''
    rv_lk = Table.read(filename, format='ascii',
                    names = ['time', 'RV', 'eRV'], units=[u.d, u.meter / u.second, u.meter / u.second])
    rv_lk = lk.LightCurve(time=rv_lk['time'], flux=rv_lk['RV'], flux_err=rv_lk['eRV'])
    return rv_lk

In [5]:
def rv_infer_sampling(rv_lk: lk.LightCurve):
    '''Infer sampling period from light curve'''
    time_diffs = rv_lk['time'][1:] - rv_lk['time'][:-1]
    return np.median(time_diffs)

## Data processing

### GTO data loading

In [6]:
# Load GTO data table:
gto = pd.read_csv(GTO_FILE, sep=',', decimal='.')
gto.head(5)

,Karmn,Name,Comp,GJ,RA_J2016_deg,DE_J2016_deg,RA_J2000,DE_J2000,l_J2016_deg,b_J2016_deg,...,T0_PG_RV,e_T0_PG_RV,offset_PG_RV,e_offset_PG_RV,FAP_PG_RV,valid_PG_RV,error_PG_RV,elapsed_time_PG_RV,fits_file_RV,fig_file_RV
0,J23585+076,Wolf 1051,AB,4383,359.636420,7.656947,23:58:32.65,+07:39:30.1,100.839686,-52.931049,...,2.457588e+06,0.000316,115.722817,156.608198,0.000103,1.0,NaN,12.416590,../data/CARMENES_GTO_RVs_PGs/J23585+076_RV_PG....,../data/CARMENES_GTO_RVs_PGs/figures/J23585+07...
1,J23556-061,GJ 912,AB,912,358.913617,-6.144283,23:55:39.78,-06:08:33.4,88.129933,-65.175491,...,2.457594e+06,0.000186,-318.903353,39.740539,0.388162,1.0,NaN,12.882091,../data/CARMENES_GTO_RVs_PGs/J23556-061_RV_PG....,../data/CARMENES_GTO_RVs_PGs/figures/J23556-06...
2,J23548+385,RX J2354.8+3831,-,NaN,358.713658,38.526340,23:54:51.46,+38:31:36.2,110.941908,-23.024449,...,2.457593e+06,0.000215,-30.744512,2.841462,0.572330,1.0,NaN,11.376183,../data/CARMENES_GTO_RVs_PGs/J23548+385_RV_PG....,../data/CARMENES_GTO_RVs_PGs/figures/J23548+38...
3,J23505-095,LP 763-012,-,4367,357.634705,-9.560964,23:50:31.64,-09:33:32.7,80.777067,-67.303426,...,2.457595e+06,0.000766,-0.832199,0.426622,0.361327,1.0,NaN,12.862077,../data/CARMENES_GTO_RVs_PGs/J23505-095_RV_PG....,../data/CARMENES_GTO_RVs_PGs/figures/J23505-09...
4,J23492+024,BR Psc,-,908,357.306604,2.396918,23:49:12.53,+02:24:04.4,93.567467,-56.885396,...,2.457395e+06,0.005702,-2.146832,0.167937,0.020497,1.0,NaN,20.084430,../data/CARMENES_GTO_RVs_PGs/J23492+024_RV_PG....,../data/CARMENES_GTO_RVs_PGs/figures/J23492+02...


In [7]:
print(list(gto.columns))

['Karmn', 'Name', 'Comp', 'GJ', 'RA_J2016_deg', 'DE_J2016_deg', 'RA_J2000', 'DE_J2000', 'l_J2016_deg', 'b_J2016_deg', 'Ref01', 'SpT', 'SpTnum', 'Ref02', 'Teff_K', 'eTeff_K', 'logg', 'elogg', '[Fe/H]', 'e[Fe/H]', 'Ref03', 'L_Lsol', 'eL_Lsol', 'Ref04', 'R_Rsol', 'eR_Rsol', 'Ref05', 'M_Msol', 'eM_Msol', 'Ref06', 'muRA_masa-1', 'emuRA_masa-1', 'muDE_masa-1', 'emuDE_masa-1', 'Ref07', 'pi_mas', 'epi_mas', 'Ref08', 'd_pc', 'ed_pc', 'Ref09', 'Vr_kms-1', 'eVr_kms-1', 'Ref10', 'ruwe', 'Ref11', 'U_kms-1', 'eU_kms-1', 'V_kms-1', 'eV_kms-1', 'W_kms-1', 'eW_kms-1', 'Ref12', 'sa_m/s/a', 'esa_m/s/a', 'Ref13', 'SKG', 'Ref14', 'SKG_lit', 'Ref14_lit', 'Pop', 'Ref15', 'vsini_flag', 'vsini_kms-1', 'evsini_kms-1', 'Ref16', 'P_d', 'eP_d', 'Ref17', 'pEWHalpha_A', 'epEWHalpha_A', 'Ref18', 'log(LHalpha/Lbol)', 'elog(LHalpha/Lbol)', 'Ref19', '1RXS', 'CRT_s-1', 'eCRT_s-1', 'HR1', 'eHR1', 'HR2', 'eHR2', 'Flux_X_E-13_ergcm-2s-1', 'eFlux_X_E-13_ergcm-2s-1', 'LX/LJ', 'eLX/LJ', 'Ref20', 'Activity', 'Ref21', 'FUV_mag',

Generate the proper auxiliary columns (with the basic periodograms results).

In [8]:
# Additional columns:
if 'n_RV' in gto.columns:
    pass
else:
    gto['n_RV'] = None # Number of points in RV curve
    gto['Ps_RV'] = None # Sampling period (d)
    gto['fs_RV'] = None # Sampling frequency (d^(-1))
    gto['wmean_RV'] = None # Mean of RV
    gto['wrms_RV'] = None # RMS of RV
    gto['info_PG_RV'] = None # Information text about the PG
    gto['maxP_PG_RV'] = None # Max power value in the PG
    gto['maxSNR_PG_RV'] = None # Max power value in the PG
    gto['rms_PG_RV'] = None # RMS value in the PG residuals
    gto['f_PG_RV'] = None # Best frequency in the PG (d^(-1))
    gto['e_f_PG_RV'] = None # Error of the best frequency in the PG (d^(-1))
    gto['Pd_PG_RV'] = None # Best period in the PG (d)
    gto['e_Pd_PG_RV'] = None # Error of the best period in the PG (d)
    gto['Ph_PG_RV'] = None # Best period in the PG (hours)
    gto['e_Ph_PG_RV'] = None # Error of the best period in the PG (hours)
    gto['Pm_PG_RV'] = None # Best period in the PG (minutes)
    gto['e_Pm_PG_RV'] = None # Error of the best period in the PG (minutes)
    gto['A_PG_RV'] = None # Amplitude of the best frequency
    gto['e_A_PG_RV'] = None # Error of the amplitude of the best frequency
    gto['ph_PG_RV'] = None # Amplitude of the best frequency
    gto['e_ph_PG_RV'] = None # Error of the amplitude of the best frequency
    gto['T0_PG_RV'] = None # Reference epoch of the best frequency
    gto['e_T0_PG_RV'] = None # Error of the epoch of the best frequency
    gto['offset_PG_RV'] = None # Offset of the best frequency
    gto['e_offset_PG_RV'] = None # Error of the offset of the best frequency
    gto['FAP_PG_RV'] = None # False alarm probability
    gto['valid_PG_RV'] = None # Flag to indicate if the periodogram calculation succeeded (1) or not (0).
    gto['error_PG_RV'] = None # The error raised during processing. Empty if processing was successful.
    gto['elapsed_time_PG_RV'] = None # The time elapsed in calculation
    gto['fits_file_RV'] = None # The name of the processed file.
    gto['fig_file_RV'] = None # The name of the figure file.


### Batch processing of all RV files

In [9]:
n = len(gto)
n

363

#### Batch processing

In [10]:
# Batch processing:
lapse_list = []
median_lapse = None
# NOTE THE FOR LOOP IS SEPARATED INTO SEVERAL "BATCHES", SO AS TO PREVENT POTENTIAL MEMORY PROBLEMS
# OR TOO MUCH TIME ELAPSED
#for i in range(0, 3):
#for i in range(0, 99): # Record 99 fails and takes too long to complete.
#for i in range(100, 200): # Record 99 fails and takes too long to complete.
#for i in range(200, 273): # Record 273 fails and gets stuck.
#for i in range(274, 344): # Records 273 and 344 fail and get stuck.
for i in range(345, len(gto)): # Record 344 fails and gets stuck.
    start_time = time.time()
    # Names:
    karmn = gto.loc[i, 'Karmn'] # KARMENES name
    commn = gto.loc[i, 'Name'] # Common name
    tic_id = str(gto.loc[i, 'TIC_id']) # TESS TIC identifier
    print("Record: %d, started at %s"
          %(i, time.strftime('%d/%m/%Y, %H:%M:%S', time.localtime(start_time))))
    if median_lapse is None:
        print("Previous median lapse time: %s" %median_lapse)
    else:
        print("Previous median lapse time: %.2f seconds" %median_lapse)
    print("Processing %s star..." %karmn)
    try:
        # LOAD RV FILE:
        rv_file = gto.loc[i, 'rv_file']
        print("filename: %s" %rv_file)
        rv_lk = rv_load(rv_file)
        
        # GENERATE PERIODOGRAM:
        gls = Gls((rv_lk['time'].value, rv_lk['flux'].value, rv_lk['flux_err'].value),
              fbeg=FBEG, fend=FEND, Pbeg=PBEG, Pend=PEND, ofac=OFAC, hifac=HIFAC, freq=FREQ,
              norm=NORM, ls=LS, fast=FAST, verbose=VERBOSE)
        
        # GENERATE THE FITS FILE:
        # Prepare the fits primary HDU (only header):
        primary_header = fits.Header()
        primary_header['OBJECT'] = (karmn, "KARMENES target name")
        primary_header['NAME'] = (commn, "Object common name")
        primary_header['TIC'] = (tic_id, "Object TESS identifier")
        primary_header['RA_J2000'] = ("00:05:10.89", "Object right ascension (J2000)")
        primary_header['DE_J2000'] = ("+45:47:11.6", "Object declination (J2000)")
        primary_header['SPTYPE'] = ("M1.0 V", "Spectral type")
        primary_header['TEFF_K'] = (3773, "Effective temperature in Kelvin")
        primary_header['LOGG'] = (5.07, "Logarithm of surface gravity")
        primary_header['FEH'] = (-0.04, "Metallicity")
        primary_header['L_LSUN'] = (0.0436229, "Luminosity in Solar luminosities")
        primary_header['R_RSUN'] = (0.48881, "Radius in Solar radii")
        primary_header['M_MSUN'] = (0.4918, "Mass in Solar masses")
        primary_header['D_PC'] = (11.50352803, "Distance in parsec")
        hdu_primary = fits.PrimaryHDU(header=primary_header)

        # Prepare the RV HDU:
        hdu_rv = fits.table_to_hdu(QTable(rv_lk.to_table()))
        hdu_rv.name = "RV_CURVE"
        psample = rv_infer_sampling(rv_lk).value
        fsample = 1.0 / psample
        fnyq = 2.0 * fsample
        freq_units = u.d ** (-1)
        hdu_rv.header['OBJECT'] = (karmn, "KARMENES target name")
        hdu_rv.header['PUNIT'] = u.d.to_string(format='fits')
        hdu_rv.header['FUNIT'] = freq_units.to_string(format='fits')
        hdu_rv.header['RVPOINTS'] = (gls.N, "Number of points in the RV curve")
        hdu_rv.header['AVGFLUX'] = (gls._Y, "Average flux of RV curve")
        hdu_rv.header['RMSFLUX'] = (gls._YY, "Flux RMS of RV curve")
        hdu_rv.header['PSAMPLE'] = (psample, "Inferred cadence in RV curve")
        hdu_rv.header['FSAMPLE'] = (fsample, "Inferred sampling frequency in RV curve")
        hdu_rv.header['FNYQUIST'] = (fnyq, "Calculated Nyquist frequency value")
                
        # Prepare the PG HDU:
        hdu_pg = fits.table_to_hdu(
            QTable(data=[gls.freq, gls.power], names=['freq', 'power'], 
                   units=[1.0 / u.d, (u.m / u.s) ** 2]))
        hdu_pg.name = "GLS_PG"
        fpoints = len(gls.f)
        fres = (gls.fend - gls.fbeg) / (fpoints - 1)
        hdu_pg.header['OBJECT'] = (karmn, "KARMENES target name")
        hdu_pg.header['FUNIT'] = (freq_units.to_string(format='fits'), "Unit for frequencies")
        hdu_pg.header['PUNIT'] = (u.d.to_string(format='fits'), "Unit for periods")
        hdu_pg.header['PK_FREQ'] = (gls.best['f'], "Frequency of the peak in periodogram")
        hdu_pg.header['PK_POW'] = (gls.pmax, "Power of the peak in periodogram")
        hdu_pg.header['PK_SNR'] = (gls.best['amp'] / gls.rms, "SNR of the peak in periodogram")
        hdu_pg.header['PK_FAP'] = (gls.FAP(Pn=None), "FAP of the peak in periodogram")
        hdu_pg.header['RES_RMS'] = (gls.rms, "RMS of residuals in periodogram")
        hdu_pg.header['FSAMPLE'] = (fsample, "Inferred sampling frequency in RV curve")
        hdu_pg.header['FNYQUIST'] = (fnyq, "Calculated Nyquist frequency value")
        hdu_pg.header['FPOINTS'] = (fpoints, "Number of points in periodogram")
        hdu_pg.header['FBEG'] = (gls.fbeg, "Start frequency in periodogram")
        hdu_pg.header['FEND'] = (gls.fend, "End frequency in periodogram")
        hdu_pg.header['FRES'] = (fres, "Frequency resolution in periodogram")
        hdu_pg.header['F'] = (gls.best['f'], "Peak best estimate: frequency")
        hdu_pg.header['E_F'] = (gls.best['e_f'], "Peak best estimate: frequency error")
        hdu_pg.header['P'] = (gls.best['P'], "Peak best estimate: period")
        hdu_pg.header['E_P'] = (gls.best['e_P'], "Peak best estimate: period error")
        hdu_pg.header['A'] = (gls.best['amp'], "Peak best estimate: amplitude")
        hdu_pg.header['E_A'] = (gls.best['e_amp'], "Peak best estimate: amplitude error")
        hdu_pg.header['PH'] = (gls.best['ph'], "Peak best estimate: phase")
        hdu_pg.header['E_PH'] = (gls.best['e_ph'], "Peak best estimate: phase error")
        hdu_pg.header['T0'] = (gls.best['T0'], "Peak best estimate: frequency")
        hdu_pg.header['E_T0'] = (gls.best['e_T0'], "Peak best estimate: frequency error")
        hdu_pg.header['OFF'] = (gls.best['offset'], "Peak best estimate: offset")
        hdu_pg.header['E_OFF'] = (gls.best['e_offset'], "Peak best estimate: offset error")
        hdu_pg.header['OFAC'] = (gls.ofac, "Setup: oversampling factor")
        hdu_pg.header['HIFAC'] = (gls.ofac, "Setup: maximum frequency factor")
        hdu_pg.header['NORM'] = (gls.norm, "Setup: normalization type")
        hdu_pg.header['LS'] = (gls.ls, "Setup: conventional Lomb-Scargle calculation")
        hdu_pg.header['FAST'] = (gls.fast, "Setup: fast evaluation, recursive trigonometric")
        
        # Create and save the fits file:
        fits_filename = OUT_PROCESSED_FOLDER + karmn + "_RV_PG.fits"
        hdul = fits.HDUList([hdu_primary, hdu_rv, hdu_pg])
        hdul.writeto(fits_filename, overwrite=True)
        
        # Delete all the HDUs:
        del hdul
        del hdu_primary
        del hdu_rv
        del hdu_pg

        # FILL IN THE DATA IN THE GTO TABLE:
        gto.loc[i, 'n_RV'] = gls.N
        gto.loc[i, 'Ps_RV'] = psample
        gto.loc[i, 'fs_RV'] = fsample
        gto.loc[i, 'wmean_RV'] = gls._Y
        gto.loc[i, 'wrms_RV'] = gls._YY
        gto.loc[i, 'info_PG_RV'] = gls.info(stdout=False)
        gto.loc[i, 'maxP_PG_RV'] = gls.power.max()
        gto.loc[i, 'maxSNR_PG_RV'] = gls.best['amp'] / gls.rms
        gto.loc[i, 'rms_PG_RV'] = gls.rms
        gto.loc[i, 'f_PG_RV'] = gls.best['f']
        gto.loc[i, 'e_f_PG_RV'] = gls.best['e_f']
        gto.loc[i, 'Pd_PG_RV'] = gls.best['P']
        gto.loc[i, 'e_Pd_PG_RV'] = gls.best['e_P']
        gto.loc[i, 'Ph_PG_RV'] = 24.0 * gls.best['P']
        gto.loc[i, 'e_Ph_PG_RV'] = 24.0 * gls.best['e_P']
        gto.loc[i, 'Pm_PG_RV'] = 24.0 * 60.0 * gls.best['P']
        gto.loc[i, 'e_Pm_PG_RV'] = 24.0 * 60.0 * gls.best['e_P']
        gto.loc[i, 'A_PG_RV'] = gls.best['amp']
        gto.loc[i, 'e_A_PG_RV'] = gls.best['e_amp']
        gto.loc[i, 'ph_PG_RV'] = gls.best['ph']
        gto.loc[i, 'e_ph_PG_RV'] = gls.best['e_ph']
        gto.loc[i, 'T0_PG_RV'] = gls.best['T0']
        gto.loc[i, 'e_T0_PG_RV'] = gls.best['e_T0']
        gto.loc[i, 'offset_PG_RV'] = gls.best['offset']
        gto.loc[i, 'e_offset_PG_RV'] = gls.best['e_offset']
        gto.loc[i, 'FAP_PG_RV'] = gls.FAP(Pn=None)
        
        # GENERATE THE FIGURE:
        fig = gls.plot(block=False, period=False,
                       fap=FAP_LEVELS_PLOT, gls=True, data=True, residuals=True)
        figure_title = "%s, P=%.4f [min], FAP=%.4f%%" \
            %(karmn, gto.loc[i, 'Pm_PG_RV'], 100.0 * gto.loc[i, 'FAP_PG_RV'])
        fig.suptitle(figure_title, fontdict = {'fontsize' : 36})
        fig.tight_layout()
        # Save the figure to disk:
        fig_file = OUT_IMG_FOLDER + karmn + "_RV_PG.png"
        fig.savefig(fig_file)
        plt.close() # Prevent the figure from showing.

        # SET THE RECORD CALCULATION AS VALID AND STORE THE REULTING FILENAMES
        gto.loc[i, 'valid_PG_RV'] = 1
        gto.loc[i, 'error_PG_RV'] = ""
        gto.loc[i, 'fits_file_RV'] = fits_filename
        gto.loc[i, 'fig_file_RV'] = fig_file

        # UPDATE THE AVERAGE RECORD PROCESSING TIME:
        lapse = time.time() - start_time
        lapse_list.append(lapse)
        median_lapse = np.nanmedian(lapse_list)
        gto.loc[i, 'elapsed_time_PG_RV'] = lapse
        
        # SAVE THE UPDATED GTO TABLE TO DISK:
        gto.to_csv(GTO_FILE, sep=',', decimal='.', index=False)
        
        # Report successful execution:
        print("Elapsed time: %.2f seconds" %lapse)
        print("... SUCCESS.")
        
    except Exception as e:
        # Some error happened, establish the record as not valid and record the error:
        error = "*** Some ERROR happened with record #%d, %s star. Error=%s" %(i, karmn, e)
        print(error)
        gto.loc[i, 'valid_PG_RV'] = 0
        gto.loc[i, 'error_PG_RV'] = e
        
        # Try to update the record, and save the file:
        try:
            # UPDATE THE AVERAGE RECORD PROCESSING TIME:
            lapse = time.time() - start_time
            lapse_list.append(lapse)
            median_lapse = np.nanmedian(lapse_list)
            gto.loc[i, 'elapsed_time_PG_RV'] = lapse
            print("Elapsed time: %.2f seconds" %lapse)

            # SAVE THE UPDATED GTO TABLE TO DISK:
            gto.to_csv(GTO_FILE, sep=',', decimal='.', index=False)
        except Exception as e2:
            error_2 = "*** Additional ERROR happened with record #%d, %s star. Error=%s" %(i, karmn, str(e2))
            print(error)
            gto.loc[i, 'error_PG_RV'] = gto.loc[i, 'error_PG_RV'] + "/" + str(e2)



Record: 345, started at 26/03/2022, 10:13:45
Previous median lapse time: None
Processing J01339-176 star...
filename: ../data/CARMENES_GTO_RVs/J01339-176.dat


c:\users\ciroe\gls.py:642: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


Elapsed time: 13.37 seconds
... SUCCESS.
Record: 346, started at 26/03/2022, 10:13:59
Previous median lapse time: 13.37 seconds
Processing J01125-169 star...
filename: ../data/CARMENES_GTO_RVs/J01125-169.dat


c:\users\ciroe\gls.py:642: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


Elapsed time: 13.66 seconds
... SUCCESS.
Record: 347, started at 26/03/2022, 10:14:12
Previous median lapse time: 13.52 seconds
Processing J01066+192 star...
filename: ../data/CARMENES_GTO_RVs/J01066+192.dat


...

Elapsed time: 17.00 seconds
... SUCCESS.
Record: 361, started at 26/03/2022, 10:17:21
Previous median lapse time: 13.22 seconds
Processing J00067-075 star...
filename: ../data/CARMENES_GTO_RVs/J00067-075.dat


c:\users\ciroe\gls.py:642: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


Elapsed time: 13.37 seconds
... SUCCESS.
Record: 362, started at 26/03/2022, 10:17:34
Previous median lapse time: 13.37 seconds
Processing J00051+457 star...
filename: ../data/CARMENES_GTO_RVs/J00051+457.dat


c:\users\ciroe\gls.py:642: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


Elapsed time: 12.27 seconds
... SUCCESS.


## Review a given file

In [11]:
object_name = "J23556-061"

In [12]:
read_hdul = fits.open(OUT_PROCESSED_FOLDER + object_name + "_RV_PG.fits", mode='update', memmap=False)
read_hdul

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x00000140A428B3D0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x000001409E073670>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x00000140A436CC70>]

In [13]:
read_hdul.info()

Filename: ../data/CARMENES_GTO_RVs_PGs/J23556-061_RV_PG.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      17   ()      
  1  RV_CURVE      1 BinTableHDU     33   30R x 3C   [2D, D, D]   
  2  GLS_PG        1 BinTableHDU     45   1000001R x 2C   [D, D]   


### Retrieve basic information

In [14]:
read_hdul['PRIMARY'].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
OBJECT  = 'J23556-061'         / KARMENES target name                           
NAME    = 'GJ 912  '           / Object common name                             
TIC     = '9759680 '           / Object TESS identifier                         
RA_J2000= '00:05:10.89'        / Object right ascension (J2000)                 
DE_J2000= '+45:47:11.6'        / Object declination (J2000)                     
SPTYPE  = 'M1.0 V  '           / Spectral type                                  
TEFF_K  =                 3773 / Effective temperature in Kelvin                
LOGG    =                 5.07 / Logarithm of surface gravity                   
FEH     =                -0.

### Retrieve RV curve information

In [15]:
read_hdul['RV_CURVE']

In [16]:
read_hdul['RV_CURVE'].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   32 / length of dimension 1                          
NAXIS2  =                   30 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    3 / number of table fields                         
TIMESYS = 'UTC     '           / Default time scale                             
JDREF   =                  0.0 / Time columns are jd = jd1 + jd2                
TREFPOS = 'TOPOCENTER'         / Time reference position                        
TTYPE1  = 'time    '                                                            
TFORM1  = '2D      '        

In [17]:
read_hdul['RV_CURVE'].data[:10]

FITS_rec([([ 2.457594e+06, -3.412100e-01], 195.56986007, 1.85112283),
          ([ 2.457596e+06, -3.316500e-01], 192.46123284, 1.88503475),
          ([ 2.457605e+06, -3.590500e-01], 189.27936195, 1.72742755),
          ([ 2.457609e+06, -3.552200e-01], 186.0802168 , 1.83573092),
          ([ 2.457626e+06, -3.963500e-01], 178.80402101, 1.93055664),
          ([ 2.457635e+06, -4.279800e-01], 166.50448593, 2.33815054),
          ([ 2.457645e+06, -4.587700e-01], 166.28886297, 2.22307642),
          ([ 2.457673e+06, -4.852100e-01], 162.55695165, 1.66384634),
          ([ 2.457753e+06,  2.839400e-01], 134.57988318, 1.78942741),
          ([ 2.457932e+06, -3.602700e-01],  46.79777861, 2.11953017)],
         dtype=(numpy.record, [('time', '>f8', (2,)), ('flux', '>f8'), ('flux_err', '>f8')]))

In [18]:
read_rv_table = QTable.read(read_hdul['RV_CURVE'])
read_rv_table

time [2],flux,flux_err
d,m / s,m / s
float64,float64,float64
2457594.0 .. -0.3412100002169609,195.569860066,1.85112283278
2457596.0 .. -0.3316500000655651,192.461232841,1.88503475377
2457605.0 .. -0.35905000008642673,189.279361954,1.72742754679
2457609.0 .. -0.35521999979391694,186.080216805,1.83573091694
2457626.0 .. -0.3963500000536442,178.804021008,1.93055664008
2457635.0 .. -0.42798000015318394,166.504485934,2.33815054118
2457645.0 .. -0.458769999910146,166.288862972,2.22307641691
2457673.0 .. -0.4852100000716746,162.556951648,1.66384634491


### Retrieve periodogram information

In [19]:
read_hdul['GLS_PG']

In [20]:
read_hdul['GLS_PG'].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   16 / length of dimension 1                          
NAXIS2  =              1000001 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    2 / number of table fields                         
TTYPE1  = 'freq    '                                                            
TFORM1  = 'D       '                                                            
TUNIT1  = 'd-1     '                                                            
TTYPE2  = 'power   '                                                            
TFORM2  = 'D       '        

In [21]:
read_hdul['GLS_PG'].data[:10]

FITS_rec([(4.8      , 0.12799914), (4.8001392, 0.09985237),
          (4.8002784, 0.06952694), (4.8004176, 0.06138789),
          (4.8005568, 0.03964894), (4.800696 , 0.01273945),
          (4.8008352, 0.00875169), (4.8009744, 0.05022524),
          (4.8011136, 0.06408642), (4.8012528, 0.02046232)],
         dtype=(numpy.record, [('freq', '>f8'), ('power', '>f8')]))

In [22]:
read_pg_table = QTable.read(read_hdul['GLS_PG'])
read_pg_table

freq,power
1 / d,m2 / s2
float64,float64
4.8,0.12799913602578103
4.800139199999999,0.09985237299792721
4.8002784,0.06952694083381224
4.800417599999999,0.06138788996233141
4.8005568,0.03964893838246647
4.800695999999999,0.012739452471143798
4.8008352,0.00875169139046003
4.800974399999999,0.050225237454027974


In [23]:
read_pg_table['freq'][:-1]

<Quantity [  4.8      ,   4.8001392,   4.8002784, ..., 143.9995824,
           143.9997216, 143.9998608] 1 / d>

In [24]:
diff_frecs = read_pg_table['freq'][:-1] - read_pg_table['freq'][1:]
diff_frecs

<Quantity [-0.0001392, -0.0001392, -0.0001392, ..., -0.0001392, -0.0001392,
           -0.0001392] 1 / d>

### Delete read information

In [25]:
read_hdul.close()

In [26]:
del read_hdul

# Summary

**OBSERVATIONS AND CONCLUSIONS:**
- We have completed the basic GLS periodogram calculation for the radial velocity (RV) curves of GTO objetcs, and stored the results.
- Some of the objects yielded errors, and some others yielded warnings. Those objects should be analized individually.